# <center> Million Songs Analysis</center>

### Starting the Spark session

In [1]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("local[1]") \
        .appName("Project_19")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()

        
        
# Old API (RDD)
spark_context = spark_session.sparkContext

### Importations

In [292]:
import h5py
import io, time
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id, col, when

ImportError: cannot import name 'otherwise'

### Test


In [137]:
songs = h5py.File('b.h5') # type = HDF5 file.

print(f'keys are {list(songs.values())}')
group_analysis = songs['metadata'] # type: HDF5 group.
print()
print(group_analysis.keys()) # AttributeManager -- these are like a small metadata dictionary attached anywhere in the tree.

#bars_start = songs['analysis']['bars_start'] # Dataset.

# Lets look at a more complex table elsewhere in the tree...

songs = songs['metadata']['songs'] # Dataset.

# This dataset has rows and columns. Lets get the first row...
song_row = songs.value
print()
# print the column names
print(song_row.dtype.names)
print(song_row)
print(song_row.dtype.names[3])
print(song_row[0][3])
#print([song.decode("ASCII") for song in song_row])


keys are [<HDF5 group "/analysis" (16 members)>, <HDF5 group "/metadata" (5 members)>, <HDF5 group "/musicbrainz" (3 members)>]

<KeysViewHDF5 ['artist_terms', 'artist_terms_freq', 'artist_terms_weight', 'similar_artists', 'songs']>

('analyzer_version', 'artist_7digitalid', 'artist_familiarity', 'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_mbid', 'artist_name', 'artist_playmeid', 'genre', 'idx_artist_terms', 'idx_similar_artists', 'release', 'release_7digitalid', 'song_hotttnesss', 'song_id', 'title', 'track_7digitalid')
[(b'', 165270, 0.58179377, 0.40199754, b'ARD7TVE1187B99BFB1', nan, b'California - LA', nan, b'e77e51a5-4761-45b3-9847-2051f811e366', b'Casual', 4479, b'', 0, 0, b'Fear Itself', 300848, 0.60211999, b'SOMZWCG12A8C13C480', b"I Didn't Mean To", 3401791)]
artist_hotttnesss
0.4019975433642836


/home/ubuntu/.local/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [158]:
songs = h5py.File('b.h5')
songs = songs["metadata"]
print(songs['artist_terms'][:])
print(songs['artist_terms_weight'][:])


[b'hip hop' b'underground rap' b'g funk' b'alternative rap' b'gothic rock'
 b'west coast rap' b'rap' b'club dance' b'singer-songwriter' b'chill-out'
 b'underground hip hop' b'rock' b'gothic' b'san francisco bay area'
 b'indie' b'american' b'punk' b'california' b'industrial' b'new york'
 b'90s' b'latin' b'spanish' b'dark' b'ebm' b'underground' b'deathrock'
 b'west coast' b'san francisco' b'producer' b'oakland' b'catalan'
 b'barcelona' b'doomsdope' b'norcal' b'west coast hip hop'
 b'alternative rock']
[1.         0.89793596 0.88426185 0.84262975 0.84256301 0.83239282
 0.82577707 0.79859195 0.7431759  0.73850237 0.72505245 0.71389955
 0.67049417 0.65697231 0.65105613 0.65105612 0.65105597 0.65105592
 0.65105547 0.65105532 0.65105508 0.65105506 0.65105461 0.65105427
 0.65105376 0.65104997 0.6364043  0.63334971 0.61973455 0.61889383
 0.61419433 0.59579116 0.56220197 0.55067233 0.52897541 0.49021215
 0.38341077]


### Loading the data into RDDs

In [167]:
#rdd = spark_context.binaryFiles("hdfs://host-192-168-1-153-ldsa:9000/millionsongs/data/A/B/*")
rdd = spark_context.binaryFiles("/mnt/ms/data/A/B/*")

def f(x):
    with h5py.File(io.BytesIO(x[1])) as f:
        
        f_meta = f['metadata']["songs"]
        f_analys = f['analysis']["songs"]
        f_brainz = f['musicbrainz']["songs"]
        
        artist_terms = [ element.decode() for element in f['metadata']['artist_terms'][:] ]
        similar_artists = [ element.decode() for element in f['metadata']['similar_artists'][:]  ]
        return ((f_meta[0][-3].decode(), f_meta[0][-2].decode(),
                f_meta[0][9].decode(), f_meta[0][4].decode(),
                f_meta[0][-6].decode(),
                float(f_analys[0][3]), float(f_meta[0][-4]),
                float(f_analys[0][2]), float(f_analys[0][23]), 
                float(f_analys[0][-4]), int(f_brainz[0][1])), 
               
                (f_meta[0][4].decode(), f_meta[0][9].decode(),
                f_meta[0][6].decode(), float(f_meta[0][3]),
                artist_terms, similar_artists))

rdd = rdd.map(f)
rdd_songs = rdd.map(lambda x : x[0])
rdd_artists = rdd.map(lambda x : x[1])

### Turning RDDs to Data frames

*  Table Songs

In [160]:
attributes = rdd_songs.map(lambda p: Row(p[0], p[1], p[2], p[3], p[4], p[5], p[6], p[7], p[8], p[9], p[10]))
fields = [StructField("song_id", StringType(), True), StructField("title", StringType(), True), 
          StructField("artist_name", StringType(), True), StructField("artist_id", StringType(), True),
          StructField("release_album", StringType(), True), 
          StructField("duration", FloatType(), True), StructField("hotness", FloatType(), True), 
          StructField("danceability", FloatType(), True), StructField("loudness", FloatType(), True), 
          StructField("tempo", FloatType(), True), StructField("year", IntegerType())]
schema = StructType(fields)

df_songs = spark_session.createDataFrame(attributes, schema)
df_songs.show()

+------------------+--------------------+--------------------+------------------+--------------------+---------+----------+------------+--------+-------+----+
|           song_id|               title|         artist_name|         artist_id|       release_album| duration|   hotness|danceability|loudness|  tempo|year|
+------------------+--------------------+--------------------+------------------+--------------------+---------+----------+------------+--------+-------+----+
|SOPLQMB12AC4686313|Girl Of Mysteriou...|           MARC COHN|ARWRVSR1187FB575FB|    Burning The Daze| 257.5669|       0.0|         0.0| -13.107|102.073|1998|
|SOFJWKK12AB01826CD|The Cookie Bakers...|      Laurie Berkner|ARC1CJB1187B99FCA4|           Buzz Buzz|233.45587|       0.0|         0.0| -16.263| 144.71|2001|
|SOHNDZB12A8C13EDB8|      Shooting Stars|           Cauterize|ARF0COO1187B99B9EF|    So Far From Real|240.43057|0.71432906|         0.0|  -4.714| 97.148|2003|
|SOAPGDK12A6D4FAF5E|        The Blessing|     

* Table artists

In [ ]:
attributes = rdd_artists.map(lambda p: Row(p[0], p[1], p[2], p[3], p[4], p[5]))
fields = [StructField("id", StringType(), True), StructField("name", StringType(), True), 
          StructField("location", StringType(), True), StructField("hotness", FloatType(), True),
          StructField("terms", ArrayType(StringType()), True), 
          StructField("similar_artists", ArrayType(StringType()), True)]
schema = StructType(fields)

df_artists = spark_session.createDataFrame(attributes, schema)\
                          .dropDuplicates()
df_artists.show()

### Saving data frames in csv files

In [212]:
df_artists = df_artists.withColumn("similar_artists", df_artists.similar_artists.cast(StringType()))\
                       .withColumn("terms", df_artists.terms.cast(StringType()))

In [213]:
df_artists.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hotness: float (nullable = true)
 |-- terms: string (nullable = true)
 |-- similar_artists: string (nullable = true)



In [240]:
df_songs.write.format('com.databricks.spark.csv')\
        .option("header", "true")\
        .save('MySongs.csv')


In [245]:
df_artists.write.format('com.databricks.spark.csv')\
        .option("header", "true")\
        .save('MyArtists.csv')


In [252]:
ddf = spark_session.read.csv('MySongs.csv/', header = 'true', inferSchema='true')\
                   

In [256]:
dff = spark_session.read.csv("MyArtists.csv/", header = 'true', inferSchema='true')

In [283]:
dff.withColumn("similar_artists", dff["similar_artists"].split(",").cast(ArrayType(StringType())))

TypeError: 'Column' object is not callable

In [278]:
"[azzaz, zezez,ezrzrz]".split(',')

['[azzaz', ' zezez', 'ezrzrz]']

In [297]:
dff = dff.withColumn("similar_artists", col("similar_artists".split(",")))

Py4JError: An error occurred while calling z:org.apache.spark.sql.functions.col. Trace:
py4j.Py4JException: Method col([class java.util.ArrayList]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [257]:
dff.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hotness: double (nullable = true)
 |-- terms: string (nullable = true)
 |-- similar_artists: string (nullable = true)



In [180]:
ss = spark_session.read.load('file:./MySongs.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

IllegalArgumentException: 'java.net.URISyntaxException: Relative path in absolute URI: file:./MySongs.csv'

In [175]:
ss.show()

AttributeError: 'RDD' object has no attribute 'show'

df_artists.where(df_artists.name.like("%MARC%")).show()

In [ ]:
df.count()

In [ ]:
# release the cores for another application!
spark_context.stop()